In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import re
from typing import Iterable
import os
import pathlib
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [3]:
download_dir = pathlib.Path(os.getcwd()).joinpath('./Adel')

# Создаем объект Options для Firefox
options = webdriver.FirefoxOptions()

# Настройки профиля через set_preference
options.set_preference("browser.download.folderList", 2)  # 2 - custom location
options.set_preference("browser.download.manager.showWhenStarting", False)
options.set_preference("browser.download.dir", str(download_dir))
options.set_preference("browser.helperApps.neverAsk.saveToDisk", "application/octet-stream,application/pdf")
options.set_preference("pdfjs.disabled", True)  # Отключить просмотр PDF

# Запуск драйвера с опциями
driver = webdriver.Firefox(options=options)

In [4]:
driver.get('https://rosstat.gov.ru/dbscripts/munst/')
wait = WebDriverWait(driver, 10)
wait.until(EC.presence_of_element_located((By.TAG_NAME, 'body')))

<selenium.webdriver.remote.webelement.WebElement (session="bda44ad8-6a9e-4e17-8f95-7039c3978158", element="608aa4ee-7ede-43eb-9434-a9716c8d4e84")>

In [5]:
def get_all_cities(driver):
    '''
    Получаем все элементы <a> на странице с заданным регулярным выражением pattern
    driver должен находиться на необходимой странице сразу
    '''
    links = driver.find_elements(By.TAG_NAME,"a")
    
    # Регулярное выражение для нужного URL-паттерна
    pattern = r"http://rosstat\.gov\.ru/dbscripts/munst/munst.*/DBInet\.cgi"
    matching_urls = []

    for link in links:
        href = link.get_attribute("href")
        text = link.get_attribute('innerText')
        if href and re.match(pattern, href):
            matching_urls.append((href, text))
    return matching_urls

In [6]:
cities = get_all_cities(driver)

In [7]:
cities

[('http://rosstat.gov.ru/dbscripts/munst/munst14/DBInet.cgi',
  'Белгородская область'),
 ('http://rosstat.gov.ru/dbscripts/munst/munst15/DBInet.cgi',
  'Брянская область'),
 ('http://rosstat.gov.ru/dbscripts/munst/munst17/DBInet.cgi',
  'Владимирская область'),
 ('http://rosstat.gov.ru/dbscripts/munst/munst20/DBInet.cgi',
  'Воронежская область'),
 ('http://rosstat.gov.ru/dbscripts/munst/munst24/DBInet.cgi',
  'Ивановская область'),
 ('http://rosstat.gov.ru/dbscripts/munst/munst29/DBInet.cgi',
  'Калужская область'),
 ('http://rosstat.gov.ru/dbscripts/munst/munst34/DBInet.cgi',
  'Костромская область'),
 ('http://rosstat.gov.ru/dbscripts/munst/munst38/DBInet.cgi',
  'Курская область'),
 ('http://rosstat.gov.ru/dbscripts/munst/munst42/DBInet.cgi',
  'Липецкая область'),
 ('http://rosstat.gov.ru/dbscripts/munst/munst46/DBInet.cgi',
  'Московская область'),
 ('http://rosstat.gov.ru/dbscripts/munst/munst54/DBInet.cgi',
  'Орловская область'),
 ('http://rosstat.gov.ru/dbscripts/munst/munst

In [8]:
moscow = [el for el in filter(lambda x: "Татарстан" in x[1], cities)][0]

In [9]:
moscow

('http://rosstat.gov.ru/dbscripts/munst/munst92/DBInet.cgi',
 'Республика Татарстан')

In [10]:
while True:
    driver.get(moscow[0])
    wait = WebDriverWait(driver, 10)
    wait.until(EC.presence_of_element_located((By.TAG_NAME, 'body')))
    if not ('ssl' in driver.current_url):
        break

In [11]:
wait = WebDriverWait(driver, 10)
wait.until(EC.presence_of_element_located((By.TAG_NAME, 'body')))

<selenium.webdriver.remote.webelement.WebElement (session="bda44ad8-6a9e-4e17-8f95-7039c3978158", element="bc1f6eb2-404b-45d3-87e6-2f91f8aefdd7")>

In [12]:
def make_all_children_elements_visible(driver, parent=None):
    # Скрипт для изменения стиля элемента
    script = """
    if (arguments[0].style.display === 'none') {
        arguments[0].style.display = 'block';  // Можно использовать 'inline', 'inline-block' и т.д.
    }
    """

    # Находим все элементы на странице
    if(parent==None):
        parent = driver
    elements = parent.find_elements("xpath", "//*")

    for element in elements:
        style = element.get_attribute("style")
        if style and 'display: none' in style.lower():
            driver.execute_script(script, element)

def get_all_clickable_checkboxes_stage_1(driver):
    '''Ищет все checkboxts на первом этапе'''
    tree = driver.find_element(By.ID, 'g0').parent
    imgs = tree.find_elements(By.TAG_NAME, "img")

    result_imgs = []
    for img in imgs:
        if(img.get_attribute('src')=='https://rosstat.gov.ru/dbimages/toS.jpg'):
            result_imgs.append(img)
    return result_imgs


In [13]:
make_all_children_elements_visible(driver, )
checkboxes = get_all_clickable_checkboxes_stage_1(driver)

In [14]:
checkboxes[0].find_element(By.XPATH, "../../..").find_elements(By.TAG_NAME, 'td')[-1].get_attribute('innerText')

'Наличие основных строительных машин со сроком службы, превышающим срок амортизации'

In [15]:
# import random
# import time
# def click_on_checkboxes(checkboxes:Iterable):
#     for checkbox in checkboxes:    
#         try:
#             if(random.randint(0, 100)>50):
#                 driver.execute_script("arguments[0].scrollIntoView(true);", checkbox)
#                 checkbox.click()
#         except Exception as ex:
#             print(f"something is wrong, {ex}")
            

In [16]:
# for checkbox in checkboxes:
#     if(checkbox.get_attribute('src')=='/dbimages/toS.jpg'):
#         print(checkbox)

In [17]:
# click_on_checkboxes(checkboxes)
checkboxes[0].click()
# 

In [18]:
len(checkboxes)
#.get_attribute('innerText')

654

In [19]:
wait = WebDriverWait(driver, 10)
wait.until(EC.presence_of_element_located((By.TAG_NAME, 'body')))

<selenium.webdriver.remote.webelement.WebElement (session="bda44ad8-6a9e-4e17-8f95-7039c3978158", element="bc1f6eb2-404b-45d3-87e6-2f91f8aefdd7")>

In [20]:
def start_stage_2(driver):
    element = driver.find_element(By.ID, 'Knopka')
    element.click()

In [21]:
start_stage_2(driver)

In [ ]:
def select_at_stage_3(driver, year):
    checkboxes = driver.find_elements(By.TAG_NAME, 'input')
    god_checkbox = None
    for checkbox in checkboxes:
        if(checkbox.get_attribute('type')!='checkbox'):
            continue
           
        if( checkbox.get_attribute('name')!='god_chk' and 
           checkbox.get_attribute('checked')==None):
            driver.execute_script("arguments[0].scrollIntoView(true);", checkbox)
            checkbox.click()
        # Снимем выделение со всех годов путём перевыделения 
        if(checkbox.get_attribute('name')=='god_chk'):
            driver.execute_script("arguments[0].scrollIntoView(true);", checkbox)
            checkbox.click()
            wait = WebDriverWait(driver, 10)
            wait.until(EC.presence_of_element_located((By.TAG_NAME, 'body')))
            driver.execute_script("arguments[0].scrollIntoView(true);", checkbox)
            checkbox.click()

    god_select = driver.find_element(By.NAME, 'god')
    god_options = god_select.find_elements(By.TAG_NAME, 'option')
   



    for god_option in god_options:
        if(str(god_option.get_attribute('innerText'))==year):
            driver.execute_script("arguments[0].scrollIntoView(true);", god_option)
            # driver.execute_script("arguments[0].setAttribute('selected', '1')", god_option)
            god_option.click()
        # else:
            # driver.execute_script("arguments[0].setAttribute('selected', '0')", god_option)



In [23]:
year = '2024'

In [24]:
wait = WebDriverWait(driver, 10)
wait.until(EC.presence_of_element_located((By.TAG_NAME, 'body')))
select_at_stage_3(driver, year)

In [25]:
def show_table(driver):
    inputs = driver.find_elements(By.TAG_NAME, 'input')
    btn = [elem for elem in filter(lambda x: x.get_attribute('value')=='Показать таблицу', inputs)][0]
    btn.click()

In [26]:
old_url = driver.current_url
show_table(driver)
wait = WebDriverWait(driver, 10)
wait.until(EC.presence_of_element_located((By.TAG_NAME, 'body')))

<selenium.webdriver.remote.webelement.WebElement (session="bda44ad8-6a9e-4e17-8f95-7039c3978158", element="7d020ae0-6b22-4c20-b330-6548e764d9c2")>

In [27]:
#Если автоматически не перешло на новую вкладку сайта
if(old_url==driver.current_url):
    document_format = 'CSV'
    year_from = year
    year_to = year

    # У нас открыто новое окно. Старое уже не нужно, поэтому переходим на новое. 
    original_window = driver.current_window_handle
    for window_handle in driver.window_handles:
        if window_handle != original_window:
            driver.switch_to.window(window_handle)
            break
wait = WebDriverWait(driver, 10)
wait.until(EC.presence_of_element_located((By.TAG_NAME, 'body')))


<selenium.webdriver.remote.webelement.WebElement (session="bda44ad8-6a9e-4e17-8f95-7039c3978158", element="50676d53-3f84-425a-ad03-f16c01a863b8")>

In [28]:
def select_at_stage_4(driver, document_format, period_from, period_to):
    format_element = driver.find_element(By.NAME,'Format')
    year_from_element = driver.find_element(By.NAME,'YearFrom')
    year_to_element = driver.find_element(By.NAME,'YearTo')
    def choose_option(driver, container_element, option):
        options = container_element.find_elements(By.TAG_NAME, 'option')
        option = [elem for elem in filter(lambda x: option in x.get_attribute('innerText'), options)][0]
        driver.execute_script("arguments[0].scrollIntoView(true);", option)
        option.click()
    choose_option(driver, format_element, document_format)
    choose_option(driver, year_from_element, period_from)
    choose_option(driver, year_to_element, period_to)
    btn = driver.find_element(By.NAME, 'showtbl')
    btn.click()


In [30]:
select_at_stage_4(driver, document_format, year_from, year_to)